In [25]:
# импорт необходимых библиотек
import pandas as pd
import os
from docx import Document
from pony.orm import *
from models import db

In [26]:
# путь к файлам
PACH_DATA = "../raw_data/"

In [27]:
# вывод файлов директории "data"
data_list = os.listdir(PACH_DATA)
data_list

['1_Многоквартирные дома с технико-экономическими характеристиками.xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства (2).xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx',
 '3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx',
 '4_Виды работ по капитальному ремонту многоквартирных домов.xlsx',
 '4_Виды работ по содержанию и ремонту общего имущества многоквартирных домов.xlsx',
 '4_Виды_работ_по_капитальному_ремонту_многоквартирных_домов_с_фактом (3).xlsx',
 '5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx',
 'Объекты_модели_Группы_видов_работ_по_содержанию_и_ремонту_общего.xlsx',
 'Описание примеров данных.docx']

### Описание примеров данных

In [28]:
# загрузка данных файла 'Описание примеров данных.docx'
document = Document(PACH_DATA + data_list[-1])

In [29]:
# парсинг таблиц данных файла 'Описание примеров данных.docx'
df_tables = []
for table in document.tables:
    df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
    for i, row in enumerate(table.rows):
        for j, cell in enumerate(row.cells):
            if cell.text:
                df[i][j] = cell.text
    df_tables.append(pd.DataFrame(df[1:], columns=df[0]))

In [30]:
# количество обнаруженных таблиц
len(df_tables)

16

### Виды работ по капитальному ремонту многоквартирных домов

In [31]:
# путь к данным
data_list[6]

'4_Виды_работ_по_капитальному_ремонту_многоквартирных_домов_с_фактом (3).xlsx'

In [32]:
# описание данных
df_tables[13]

,Имя колонки,Комментарий
0,A,№ п/п
1,B,Код
2,C,Наименование
3,D,Наименование объекта общего имущества
4,E,Тип работ
5,F,Группа работ
6,G,Сокращенное наименование работы


In [33]:
# вывод наименование листов в файле "4_Виды работ по капитальному ремонту многоквартирных домов.xlsx"
xlsx_file = pd.ExcelFile(PACH_DATA + data_list[6])
sheet_names = xlsx_file.sheet_names
print(sheet_names)

['Лист1']


In [34]:
# загрузка данных
type_work_laste_df = pd.read_excel(PACH_DATA + data_list[6], sheet_name="Лист1", header=1, index_col=0)
type_work_laste_df.head()

,Код,Наименование,Наименование объекта общего имущества,Тип работ,Группа работ,Сокращенное наименование работы,Виды работ в 2022 году
№ п/п,,,,,,,
1,105,строительный контроль,Строительный контроль,Услуга,Стройконтроль,СК,NaN
2,26,"ремонт подъездов, направленный на восстановлен...",Подъезд,Работа,СМР,Подъезд,"ремонт подъездов, направленный на восстановлен..."
3,9,"ремонт фасада (замена оконных блоков, располож...",Фасад(оконные блоки),Работа,СМР,Окна,"замена оконных блоков, расположенных в помещен..."
4,104,проведение экспертизы проектной документации п...,проведение экспертизы,Услуга,СМР,проведение экспертизы,NaN
5,20,разработка проектной документации,разработка проектной документации,Услуга,СМР,ПСД,NaN


In [35]:
# типы данных
type_work_laste_df.dtypes

Код                                       int64
Наименование                             object
Наименование объекта общего имущества    object
Тип работ                                object
Группа работ                             object
Сокращенное наименование работы          object
Виды работ в 2022 году                   object
dtype: object

In [36]:
# процентное соотношение пропусков в данных
(type_work_laste_df.isna().sum() / type_work_laste_df.shape[0] * 100) \
    .sort_values(ascending=False).round(2)[:20].astype('str') + ' %'

Виды работ в 2022 году                   34.48 %
Код                                        0.0 %
Наименование                               0.0 %
Наименование объекта общего имущества      0.0 %
Тип работ                                  0.0 %
Группа работ                               0.0 %
Сокращенное наименование работы            0.0 %
dtype: object

In [37]:
# заполнение пропусков по смыслу
type_work_laste_df = type_work_laste_df.fillna("работы не проводились")

In [38]:
# признаки данных
type_work_laste_df.columns

Index(['Код', 'Наименование', 'Наименование объекта общего имущества',
       'Тип работ', 'Группа работ', 'Сокращенное наименование работы',
       'Виды работ в 2022 году'],
      dtype='object')

In [39]:
# количество уникальных значений
type_work_laste_df.nunique()

Код                                      23
Наименование                             29
Наименование объекта общего имущества    29
Тип работ                                 2
Группа работ                              5
Сокращенное наименование работы          29
Виды работ в 2022 году                   20
dtype: int64

In [40]:
# размерность данных
type_work_laste_df.shape

(29, 7)

Одному и томуже коду соответствует несколько наименований

In [41]:
# запись данных в файл
type_work_laste_df.to_csv("../data/type_work_laste_df.csv", index=False)

In [42]:
# вывод таблицы 'types_work_lastes'
db.Types_Work_Laste.select().show()

id|code_id|type_name|object_name|type_work|group_work|abbreviated|work_2022
--+-------+---------+-----------+---------+----------+-----------+---------


In [43]:
@db_session
def add_type_work_laste(code_id, type_name, object_name, type_work,
                        group_work, abbreviated, work_2022):

    """ функуия записи вида работ по капитальному ремонту 
    многоквартирных домов в таблицу 'types_work_lastes' """
    
    db.Types_Work_Laste(code_id=code_id, type_name=type_name,
                        object_name=object_name, type_work=type_work,
                        group_work=group_work, abbreviated=abbreviated,
                        work_2022=work_2022)
    # commit() will be done automatically
    # database session cache will be cleared automatically
    # database connection will be returned to the pool

In [44]:
# данные для записи в таблицу
type_work_laste = type_work_laste_df.values
type_work_laste[:2]

array([[105, 'строительный контроль', 'Строительный контроль', 'Услуга',
        'Стройконтроль', 'СК', 'работы не проводились'],
       [26,
        'ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ',
        'Подъезд', 'Работа', 'СМР', 'Подъезд',
        'ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ по капитальному ремонту общего имущества в многоквартирном доме']],
      dtype=object)

In [45]:
# запись данных в базу данных
for code_id, type_name, object_name, type_work, \
    group_work, abbreviated, work_2022 in type_work_laste:
    add_type_work_laste(code_id, type_name, object_name, type_work,
                        group_work, abbreviated, work_2022)

TransactionIntegrityError: Newly auto-generated id value 1 was already used in transaction cache for another object

In [ ]:
# проверка записи данных
db.Types_Work_Laste.select()[:10].show()

id|code_id|type_name  |object_name|type_work|group_work |abbreviated|work_2022  
--+-------+-----------+-----------+---------+-----------+-----------+-----------
1 |105    |строител...|Строител...|Услуга   |Стройкон...|СК         |работы н...
2 |26     |ремонт п...|Подъезд    |Работа   |СМР        |Подъезд    |ремонт п...
3 |9      |ремонт ф...|Фасад(ок...|Работа   |СМР        |Окна       |замена о...
4 |104    |проведен...|проведен...|Услуга   |СМР        |проведен...|работы н...
5 |20     |разработ...|разработ...|Услуга   |СМР        |ПСД        |работы н...
6 |3      |ремонт в...|Внутридо...|Работа   |СМР        |ЦО-М       |ремонт в...
7 |5      |ремонт в...|Внутридо...|Работа   |СМР        |КАН-М      |ремонт в...
8 |4      |ремонт в...|Внутридо...|Работа   |СМР        |ГВС-М      |ремонт в...
9 |4      |ремонт в...|Внутридо...|Работа   |СМР        |ХВС-М      |ремонт в...
10|53     |ремонт и...|Мусоропр...|Работа   |СМР        |Мусоропр...|ремонт м...


In [ ]:
# проверка записи данных
db.select("select count(*) from types_work_lastes")[0], len(type_work_laste)

(29, 29)

Данные записаны **успешно**